# Optimization Packages in Python

**Linear programming** is a set of techniques used in mathematical programming, sometimes called mathematical optimization, to solve systems of linear equations and inequalities while maximizing or minimizing some linear function.

**Mixed-integer linear programming** is an extension of linear programming. It handles problems in which at least one variable takes a discrete integer rather than a continuous value. Although mixed-integer problems look similar to continuous variable problems at first sight, they offer significant advantages in terms of flexibility and precision.

## Table of Contents

* [Scipy Library](#Scipy-Library)
* [PulP Library](#PulP-Library)
* [OR Tools Library](#OR-Tools-Library)
* [Pyomo](#Pyomo)
* [Other Open Source Solvers](#Other-Open-Source-Solvers)



## Scipy Library
[*Back to Table of Contents*](#Table-of-Contents)

In [1]:
# Install Package
# !pip install scipy

### Define the problem


Maximize z = x + 2y 
subject to the following constraints:
* 2x + y ≤ 20
* -4x + 5y ≤ 10
* -x + 2y ≥ -2
* -x + 5y = 15 
* x ≥ 0
* y ≥ 0

However, Transform to: \
Minimize -z = -x - 2y 
subject to the following constraints:
* 2x + y ≤ 20
* -4x + 5y ≤ 10
* x - 2y ≤ 2
* -x + 5y = 15 
* x ≥ 0
* y ≥ 0

In [2]:
from scipy.optimize import linprog

### Define inputs of solver

In [3]:
obj = [-1, -2]
#      ─┬  ─┬
#       │   └┤ Coefficient for y
#       └────┤ Coefficient for x

lhs_ineq = [[ 2,  1],  # Red constraint left side
            [-4,  5],  # Blue constraint left side
            [ 1, -2]]  # Yellow constraint left side

rhs_ineq = [20,  # Red constraint right side
            10,  # Blue constraint right side
            2]   # Yellow constraint right side

lhs_eq = [[-1, 5]]  # Green constraint left side
rhs_eq = [15]       # Green constraint right side

bnd = [(0, float("inf")),  # Bounds of x
       (0, float("inf"))]  # Bounds of y

### Solve the problem

In [4]:
opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq,
              A_eq=lhs_eq, b_eq=rhs_eq, bounds=bnd,
              method="revised simplex")
print(opt)

     con: array([0.])
     fun: -16.818181818181817
 message: 'Optimization terminated successfully.'
     nit: 3
   slack: array([ 0.        , 18.18181818,  3.36363636])
  status: 0
 success: True
       x: array([7.72727273, 4.54545455])


### Limitations of Scipy

* SciPy can’t run various external solvers.
* SciPy can’t run various external solvers.
* SciPy can’t work with integer decision variables.
* SciPy doesn’t provide classes or functions that facilitate model building. You have to define arrays and matrices, which might be a tedious and error-prone task for large problems.
* SciPy doesn’t allow you to define maximization problems directly. You must convert them to minimization problems.
* SciPy doesn’t allow you to define constraints using the greater-than-or-equal-to sign directly. You must use the less-than-or-equal-to instead.

## PulP Library

[*Back to Table of Contents*](#Table-of-Contents)

In [5]:
# Install packages
# !pip install pulp

### List of Solvers for PulP

In [6]:
import pulp
print(pulp.listSolvers())
print(pulp.listSolvers(onlyAvailable = True))

['GLPK_CMD', 'PYGLPK', 'CPLEX_CMD', 'CPLEX_PY', 'GUROBI', 'GUROBI_CMD', 'MOSEK', 'XPRESS', 'PULP_CBC_CMD', 'COIN_CMD', 'COINMP_DLL', 'CHOCO_CMD', 'MIPCL_CMD', 'SCIP_CMD']
['GLPK_CMD', 'PULP_CBC_CMD']


### Solve using Default Solver

In [7]:
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

# Create the model
model = LpProblem(name="small-problem", sense=LpMaximize)

# Initialize the decision variables
x = LpVariable(name="x", lowBound=0)
y = LpVariable(name="y", lowBound=0)

# Add the constraints to the model
model += (2 * x + y <= 20, "red_constraint")
model += (4 * x - 5 * y >= -10, "blue_constraint")
model += (-x + 2 * y >= -2, "yellow_constraint")
model += (-x + 5 * y == 15, "green_constraint")

# Add the objective function to the model
model += lpSum([x, 2 * y])
print('MODEL: ')
print(model)

# Solve the problem
status = model.solve()

# Print results
print('SOLUTION:')
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for var in model.variables():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

    
print(f'Solver: {model.solver}')

MODEL: 
small-problem:
MAXIMIZE
1*x + 2*y + 0
SUBJECT TO
red_constraint: 2 x + y <= 20

blue_constraint: 4 x - 5 y >= -10

yellow_constraint: - x + 2 y >= -2

green_constraint: - x + 5 y = 15

VARIABLES
x Continuous
y Continuous

SOLUTION:
status: 1, Optimal
objective: 16.8181817
x: 7.7272727
y: 4.5454545
red_constraint: -9.99999993922529e-08
blue_constraint: 18.181818300000003
yellow_constraint: 3.3636362999999996
green_constraint: -2.0000000233721948e-07
Solver: <pulp.apis.coin_api.PULP_CBC_CMD object at 0x0000022C18071BB0>


### Solve Using Different Solver: GLPK

* GLPK Documentation : https://www.gnu.org/software/glpk/

* Download GLPK using  conda thru the following command: `conda install -c conda-forge glpk` or visit https://anaconda.org/conda-forge/glpk

In [8]:
from pulp import GLPK

# Create the model
model = LpProblem(name="small-problem", sense=LpMaximize)

# # Initialize the decision variables and with Integer Values
# x = LpVariable(name="x", lowBound=0, cat="Integer")
# y = LpVariable(name="y", lowBound=0, cat="Integer")

# Initialize the decision variables 
x = LpVariable(name="x", lowBound=0)
y = LpVariable(name="y", lowBound=0)

# Add the constraints to the model
model += (2 * x + y <= 20, "red_constraint")
model += (4 * x - 5 * y >= -10, "blue_constraint")
model += (-x + 2 * y >= -2, "yellow_constraint")
model += (-x + 5 * y == 15, "green_constraint")

# Add the objective function to the model
model += lpSum([x, 2 * y])

# Solve the problem
status = model.solve(solver=GLPK(msg=False))

# Print results
print('SOLUTION:')
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for var in model.variables():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

print(f'Solver: {model.solver}')

SOLUTION:
status: 1, Optimal
objective: 16.81817
x: 7.72727
y: 4.54545
red_constraint: -1.0000000000509601e-05
blue_constraint: 18.181830000000005
yellow_constraint: 3.3636299999999997
green_constraint: -2.000000000279556e-05
Solver: <pulp.apis.glpk_api.GLPK_CMD object at 0x0000022C18111280>


## OR Tools Library

Link: https://github.com/google/or-tools

[*Back to Table of Contents*](#Table-of-Contents)


In [9]:
# Install Packaages
#!pip install ortools

In [10]:
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit


# Create the linear solver with the GLOP backend.
solver = pywraplp.Solver.CreateSolver('GLOP')

# Create the variables x and y.
infinity = solver.infinity()
x = solver.NumVar(lb=0, ub=infinity, name='x')
y = solver.NumVar(lb=0, ub=infinity, name='y')
print('Number of variables =', solver.NumVariables())

# Create a linear constraint
solver.Add(2 * x + y <= 20)
solver.Add(4 * x - 5 * y >= -10)
solver.Add(-x + 2 * y >= -2)
solver.Add(-x + 5 * y == 15)
print('Number of constraints =', solver.NumConstraints())

# Create the objective function.
solver.Maximize(x + 2 * y)

# Solve and Print the Problem
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    print('x =', x.solution_value())
    print('y =', y.solution_value())
else:
    print('The problem does not have an optimal solution.')

Number of variables = 2
Number of constraints = 4
Solution:
Objective value = 16.81818181818182
x = 7.72727272727273
y = 4.545454545454546


## Pyomo

[*Back to Table of Contents*](#Table-of-Contents)

In [11]:
# Install packages
# !pip install pyomo

In [12]:
from pyomo.environ import *

# Create the model object
model = ConcreteModel()

# Add the Decision Variables, Objective, and Constraints to the model object
model.x = Var(domain=NonNegativeReals)
model.y = Var(domain=NonNegativeReals)
model.profit = Objective(expr = 1*model.x + 2*model.y, sense=maximize)

model.red_constraint = Constraint(expr = 2 * model.x + model.y <= 20)
model.blue_constraint = Constraint(expr = 4 * model.x - 5 * model.y >= -10)
model.yellow_constraint = Constraint(expr = -model.x + 2 * model.y >= -2)
model.green_constraint = Constraint(expr = -model.x + 5 * model.y == 15)


# Create a solver object and solve
results = SolverFactory('glpk').solve(model)
results.write()
if results.solver.status == 'ok':
    model.pprint()

    
# display solution
print('\nProfit = ', model.profit())
print('\nDecision Variables')
print('x = ', model.x())
print('y = ', model.y())
print('\nConstraints')
print('red_constraint  = ', model.red_constraint())
print('blue_constraint = ', model.blue_constraint())
print('yellow_constraint = ', model.yellow_constraint())
print('green_constraint = ', model.green_constraint())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 16.8181818181818
  Upper bound: 16.8181818181818
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 3
  Number of nonzeros: 9
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.09146571159362793
# ----------------------------------------------------------
#   Solution Information
# ---------------

### Using Arrays

In [13]:
from pyomo.environ import *
import numpy as np

# enter data as numpy arrays
A = np.array([[2, 1], [-4, 5],[1,-2], [-1,5]]) # coefficient of constraints
b = np.array([20, 10,2, 15])           # coefficient of max value of constraints
c = np.array([1,2])                #coefficient of objective function

# set of row indices
I = range(len(A))

# set of column indices
J = range(len(A.T))

# create a model instance
model = ConcreteModel()

# create x and y variables in the model
model.x = Var(J)

# add model constraints
model.constraints = ConstraintList()
for i in I:
    model.constraints.add(sum(A[i,j]*model.x[j] for j in J) <= b[i])
    if i == len(A)-1:
        model.constraints.add(sum(A[i,j]*model.x[j] for j in J) == b[i])

# add a model objective
model.objective = Objective(expr = sum(c[j]*model.x[j] for j in J), sense=maximize)

# create a solver
solver = SolverFactory('glpk')

# solve
solver.solve(model)

# print solutions
for j in J:
    print("x[", j, "] =", model.x[j].value)

model.constraints.display()

x[ 0 ] = 7.72727272727273
x[ 1 ] = 4.54545454545454
constraints : Size=5
    Key : Lower : Body               : Upper
      1 :  None :               20.0 :  20.0
      2 :  None : -8.181818181818223 :  10.0
      3 :  None : -1.363636363636349 :   2.0
      4 :  None : 14.999999999999968 :  15.0
      5 :  15.0 : 14.999999999999968 :  15.0


## Other Open Source Solvers
* CLP & CBC : https://github.com/coin-or/COIN-OR-OptimizationSuite \
`conda install -c conda-forge ipopt` \
`conda install -c conda-forge coincbc`

* LP_SOLVE
* GEKKO: https://github.com/BYU-PRISM/GEKKO
* CVXOPT: https://github.com/cvxopt/cvxopt


[*Back to Table of Contents*](#Table-of-Contents)